<img src="./logo_UTN.svg" align="right" width="150" /> 


<br><br>
# Densidad Espectral de Potencia (PSD) del ECG

## 1. Introducción

La **Densidad Espectral de Potencia (PSD)** describe cómo se distribuye la potencia de una señal a través de sus frecuencias. En señales fisiológicas como el **ECG**, esto permite:

- Identificar componentes de la señal asociados con la frecuencia cardíaca y su variabilidad.
- Detectar ruido producido por movimientos e interferencia electromagnética.
- Comparar diferentes métodos de estimación de PSD para determinar cuál es el más estable y confiable según el tipo de señal y objetivo.

En este proyecto se analizan tres métodos:

1. **Periodograma**
2. **Welch**
3. **Blackman-Tukey**

Los tres métodos son aplicados a un segmento de ECG de aproximadamente **15.000 muestras (~2 minutos)** para equilibrar resolución y tiempo de cálculo.

---

## 2. Experimentos realizados

### 2.1 Señal ECG en el tiempo

Se grafica la señal original del ECG en función del tiempo.

**Objetivo:**  
- Tener un **referente temporal** de la señal.  
- Detectar visualmente movimientos o irregularidades.  
  Antes de hacer el análisis espectral, es fundamental observar la señal
  para asegurarse de que no haya picos inesperados, saturación o errores de adquisición.

---

### 2.2 FFT de la señal ECG

Se calcula la FFT de la señal completa y se grafica su magnitud.  
Se toma la magnitud \(|X[k]|\) para ver la intensidad de cada componente de frecuencia.

**Objetivo:**  
- Observar la **composición espectral directa** del ECG.  
- Identificar frecuencias dominantes (ej.: ritmo cardíaco, ruido).  

---

### 2.3 Periodograma

Se aplica el **periodograma ventaneado** con ventana Hamming.  

**Metodología:**  
La PSD se estima directamente de la FFT de la señal:

$$
\text{PSD}(f) = \frac{1}{N \cdot fs} \left| \text{FFT}[x[n] \cdot w[n]] \right|^2
$$

donde \( w[n] \) es la ventana (Hamming) que **suaviza los bordes** para evitar fugas espectrales.

**Objetivo:**  
- Obtener la distribución de potencia de la señal de manera **directa y sencilla**.  
- Visualizar y calcular el **ancho de banda**.

**Justificación:**  
El periodograma es **rápido y fácil de implementar**, pero **sensible al ruido**, especialmente en señales fisiológicas cortas.

**Ventajas:**
- Método simple y directo.
- Buen primer vistazo de la PSD.

**Desventajas:**
- Alta **varianza** en estimaciones.
- Puede mostrar picos falsos si la señal es corta o ruidosa.

---

### 2.4 Método de Welch

- Se divide la señal en **segmentos solapados** (2048 muestras, solapamiento 50%).  
- Se calcula el periodograma de cada segmento y se los promedia.

**Base matemática:**  

$$
\text{PSD}_{Welch}(f) = \frac{1}{K} \sum_{k=1}^{K} \text{PSD}_{segmento_k}(f)
$$

Promediando se reduce la **varianza** y se obtiene una PSD más **suave y estable**.

**Objetivo:**  
- Estimar la PSD con **menos ruido que el periodograma**.  
- Calcular ancho de banda de manera más confiable.

**Justificación:**  
Promediar varios segmentos reduce los picos aleatorios. Ideal para señales fisiológicas donde la estabilidad espectral es importante.

**Ventajas:**
- PSD más suave y estable.
- Reduce efectos de ruido.

**Desventajas:**
- Menor resolución en frecuencia si los segmentos son cortos.
- Requiere más cálculos que el periodograma simple.

---

### 2.5 Método Blackman-Tukey

- Se calcula la **autocorrelación limitada** de la señal.  
- Se aplica ventana (Hamming en este caso) sobre la autocorrelación.  
- Se calcula la FFT de la autocorrelación para obtener PSD.

**Base matemática:**  

Autocorrelación:

$$
r_{xx}[m] = \sum_{n} x[n] x[n-m]
$$

PSD a partir de autocorrelación:

$$
\text{PSD}(f) = \text{FFT}[r_{xx}[m]]
$$

**Notas adicionales:**

- El gráfico de Blackman-Tukey a veces se ve **espejado**, porque la FFT se calcula de la **autocorrelación simétrica**. Esto genera un espectro que contiene tanto las frecuencias positivas como sus reflejos (simetría alrededor de cero), por lo que solo se grafica la mitad positiva para el análisis práctico.
- La función implementada en Python se **basó en el código MATLAB del libro de Monson H. Hayes** (“Statistical Digital Signal Processing and Modeling”), que traduje a Python, manteniendo la misma lógica de autocorrelación y ventana.

**Objetivo:**  
- Obtener una PSD **suavizada**, reduciendo ruido y fluctuaciones.  
- Ideal para señales donde se quiere preservar la **forma general del espectro** más que los picos exactos.

**Justificación:**  
Útil cuando la señal es **corta o ruidosa**, ofrece una estimación más **robusta** del ancho de banda.

**Ventajas:**
- Suavidad y estabilidad.
- Menos sensible a picos aleatorios.

**Desventajas:**
- Requiere más cálculos que los métodos anteriores.
- Menor resolución si el número de retardos \(M\) es pequeño.
- Gráfico espejado si no se selecciona la mitad positiva del espectro.

---

## 3. Análisis del Ancho de Banda

- Para cada método se calcula el **rango de frecuencias donde la PSD supera el 1% del máximo**.  
- Esto indica las **frecuencias relevantes** de la señal.

**Objetivo:**  
- Comparar los métodos de PSD de forma cuantitativa.  
- Determinar qué métodos ofrecen una **estimación más consistente del ancho de banda**.

**Justificación:**  
- En ECG, frecuencias típicas:  
  - Ritmo cardíaco: 0.5–4 Hz  
  - Ruido: >50 Hz  
- Permite separar señal de ruido y evaluar la precisión de cada método.

---

## 4. Conclusiones

1. **Periodograma:**  
   - Rápido y simple.  
   - Bueno para un **primer vistazo**.  
   - Problema: varianza alta, sensible a ruido.

2. **Welch:**  
   - PSD más estable y suavizada.  
   - Reduce picos falsos y ruido.  
   - Ideal cuando se necesita **robustez estadística**.  

3. **Blackman-Tukey:**  
   - PSD muy suave y estable, gracias a autocorrelación.  
   - Mejor para señales **cortas o ruidosas**.  
   - Más costoso computacionalmente.  
   - Gráfico espejado si no se selecciona la mitad positiva del espectro.

4. **Comparación de los métodos:**  

| Método          | Resolución | Suavidad | Sensibilidad al ruido | Complejidad |
|-----------------|------------|----------|--------------------|------------|
| Periodograma    | Alta       | Baja     | Alta               | Baja       |
| Welch           | Media      | Media    | Media              | Media      |
| Blackman-Tukey  | Media-Baja | Alta     | Baja               | Alta       |

5. **Selección del método según el objetivo:**  
- **Exploración rápida:** periodograma.  
- **Estimación confiable de PSD en ECG largo:** Welch.  
- **Señales cortas o muy ruidosas:** Blackman-Tukey.  

6. **Sobre el ancho de banda:**  
- Es útil para **detectar el rango de frecuencia donde la señal es relevante**.  
- Permite filtrar ruido y comparar métodos.  
- Welch y Blackman-Tukey suelen ofrecer rangos más consistentes y menos sobreestimados que el periodograma.

---

## 5. Resumen

- **PSD = cómo se reparte la potencia de la señal entre frecuencias**.  
- **Periodograma** = directo, rápido, ruidoso.  
- **Welch** = promedio de segmentos, más estable.  
- **Blackman-Tukey** = basado en autocorrelación, muy suave, origen en Monson H. Hayes.  
- **Ancho de banda** = frecuencias “activas” de la señal.  
- Cada método tiene ventajas y limitaciones, y la elección depende de **la longitud de señal, el ruido y el objetivo del análisis**.

